In [1]:
import copy

In [2]:
from gym_electric_motor import reward_functions
import pandas as pd
import numpy as np

In [3]:
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt

In [4]:
from drmdp import feats, task
from drmdp.envs import gem

In [5]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="pos-enf", max_episode_steps=2500)
env

<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>

## Control: Current Impl - neg-enf

In [6]:
MAX_STEPS = 2500

In [7]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="esp-neg", max_episode_steps=MAX_STEPS)

In [8]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>,
 <drmdp.core.EnvMonitor at 0x3177f4510>)

In [9]:
rew_delay = task.reward_delay_distribution(
    None
)

In [10]:
rew_delay

In [11]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [12]:
feats_spec = {"name": "scale", "args": None}

In [13]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [14]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [15]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )
    
    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 858, Mean returns: -2147483725.131781;


2025-04-08 16:41:32,851 INFO root: Episode 100 mean returns: -1352915021.212380


Episode: 100; Steps: 2500, Mean returns: -1339519826.4792855;


2025-04-08 16:41:47,969 INFO root: Episode 200 mean returns: -1299227936.720370


Episode: 200; Steps: 1089, Mean returns: -1303448115.4748192;


...

2025-04-08 16:44:18,651 INFO root: Episode 300 mean returns: -1352915012.435520


Episode: 300; Steps: 825, Mean returns: -1355554775.832278;


2025-04-08 16:44:33,481 INFO root: Episode 400 mean returns: -1320702760.821772


Episode: 400; Steps: 2500, Mean returns: -1317409238.7009172;


2025-04-08 16:44:46,059 INFO root: Episode 500 mean returns: -1361504942.487619


In [16]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )
    
    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 2500, Mean returns: -360.0793115283471;


2025-04-08 16:44:50,697 INFO root: Episode 100 mean returns: -2104534074.888764


Episode: 100; Steps: 1700, Mean returns: -2104959319.9020557;


2025-04-08 16:44:54,740 INFO root: Episode 200 mean returns: -2126008907.769331


Episode: 200; Steps: 70, Mean returns: -2126115747.3446076;


...

2025-04-08 16:45:47,199 INFO root: Episode 300 mean returns: -2075900960.121192


Episode: 300; Steps: 311, Mean returns: -2076138776.64573;


2025-04-08 16:45:53,250 INFO root: Episode 400 mean returns: -2083059245.407061


Episode: 400; Steps: 74, Mean returns: -2083219904.7888603;


2025-04-08 16:45:58,518 INFO root: Episode 500 mean returns: -2087354211.417505


## Control: Current Impl - pos-enf

In [17]:
MAX_STEPS = 2500

In [18]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="pos-enf", max_episode_steps=MAX_STEPS)

In [19]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>,
 <drmdp.core.EnvMonitor at 0x32152fbd0>)

In [20]:
rew_delay = task.reward_delay_distribution(
    None
)

In [21]:
rew_delay

In [22]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [23]:
feats_spec = {"name": "scale", "args": None}

In [24]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [25]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [26]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )
    
    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 784, Mean returns: 1477.4932034641038;


2025-04-08 16:46:11,706 INFO root: Episode 100 mean returns: 2760.913909


Episode: 100; Steps: 965, Mean returns: 2751.667969795287;


2025-04-08 16:46:25,410 INFO root: Episode 200 mean returns: 2790.448689


Episode: 200; Steps: 932, Mean returns: 2784.9851995004815;


...

2025-04-08 16:49:01,006 INFO root: Episode 300 mean returns: 3030.069334


Episode: 300; Steps: 1260, Mean returns: 3027.551899253392;


2025-04-08 16:49:16,045 INFO root: Episode 400 mean returns: 3069.209935


Episode: 400; Steps: 796, Mean returns: 3065.212733892493;


2025-04-08 16:49:29,194 INFO root: Episode 500 mean returns: 3013.439115


In [27]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )
    
    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 71, Mean returns: 126.25950600482554;


2025-04-08 16:49:29,900 INFO root: Episode 100 mean returns: 122.667318


Episode: 100; Steps: 65, Mean returns: 122.63924453918166;


2025-04-08 16:49:30,605 INFO root: Episode 200 mean returns: 123.530057


Episode: 200; Steps: 78, Mean returns: 123.57698808094281;


...

2025-04-08 16:51:28,943 INFO root: Episode 300 mean returns: 1124.326917


Episode: 300; Steps: 2500, Mean returns: 1135.5550361927978;


2025-04-08 16:51:45,003 INFO root: Episode 400 mean returns: 1663.914975


Episode: 400; Steps: 2500, Mean returns: 1671.2659964353395;


2025-04-08 16:51:56,761 INFO root: Episode 500 mean returns: 1794.563905


## Control: Current Impl - default

In [28]:
MAX_STEPS = 2500

In [29]:
env = gem.make("Finite-CC-PMSM-v0", reward_fn="default", max_episode_steps=MAX_STEPS)

In [30]:
env, monitor = task.monitor_wrapper(env)
env, monitor

(<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>,
 <drmdp.core.EnvMonitor at 0x168f1d310>)

In [31]:
rew_delay = task.reward_delay_distribution(
    None
)

In [32]:
rew_delay

In [33]:
env = task.delay_wrapper(env, rew_delay)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [34]:
feats_spec = {"name": "scale", "args": None}

In [35]:
env = task.reward_mapper(
    env,
    mapping_spec={"name": "identity", "args": None},
    feats_spec=feats_spec,
)
env

<EnvMonitorWrapper<TimeLimit<GemObsAsVectorWrapper<FiniteCurrentControlPermanentMagnetSynchronousMotorEnv<Finite-CC-PMSM-v0>>>>>

In [36]:
feats_tfx = feats.create_feat_transformer(env=env, **feats_spec)
feats_tfx

In [37]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    monitor.clear()
    print("Turn:", turn)
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="uniform-random",
        base_seed=0,
    )
    
    results = task.policy_control(
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 2500, Mean returns: -374.31834074240703;


2025-04-08 16:52:12,242 INFO root: Episode 100 mean returns: -333.409958


Episode: 100; Steps: 826, Mean returns: -331.0062466509027;


2025-04-08 16:52:25,617 INFO root: Episode 200 mean returns: -320.933740


Episode: 200; Steps: 2190, Mean returns: -321.78417810681276;


...

2025-04-08 16:54:55,794 INFO root: Episode 300 mean returns: -302.563402


Episode: 300; Steps: 747, Mean returns: -301.9999958467348;


2025-04-08 16:55:09,886 INFO root: Episode 400 mean returns: -303.520430


Episode: 400; Steps: 2500, Mean returns: -303.7810244302077;


2025-04-08 16:55:23,908 INFO root: Episode 500 mean returns: -305.719185


In [38]:
lr = task.learning_rate(**{"name": "constant", "args": {"initial_lr": 0.01}})
num_episodes = 500
# Create spec using provided name and args for feature spec
for turn in range(3):
    print("Turn:", turn)
    monitor.clear()
    algorithm = task.create_algorithm(
        env=env,
        feats_transform=feats_tfx,
        delay_reward=rew_delay,
        lr=lr,
        gamma=1.0,
        epsilon=0.2,
        policy_type="markovian",
        base_seed=0,
    )
    
    results = task.policy_control(e
        env=env,
        algorithm=algorithm,
        num_episodes=num_episodes,
        monitor=monitor,
    )
    
    for episode, snapshot in enumerate(results):
        if episode % (num_episodes//5) == 0:
            print(
                f"Episode: {episode}; Steps: {snapshot.steps}, Mean returns: {np.mean(monitor.returns + [monitor.rewards]).item()};"
            )

Turn: 0
Episode: 0; Steps: 2500, Mean returns: -368.6864596061948;


2025-04-08 16:55:32,887 INFO root: Episode 100 mean returns: -162.393274


Episode: 100; Steps: 64, Mean returns: -160.8947571606243;


2025-04-08 16:55:34,852 INFO root: Episode 200 mean returns: -98.221675


Episode: 200; Steps: 69, Mean returns: -97.80391729952973;


...

2025-04-08 16:56:16,871 INFO root: Episode 300 mean returns: -83.932976


Episode: 300; Steps: 213, Mean returns: -83.85329710220161;


2025-04-08 16:56:20,250 INFO root: Episode 400 mean returns: -76.420817


Episode: 400; Steps: 96, Mean returns: -76.31705909818169;


2025-04-08 16:56:23,398 INFO root: Episode 500 mean returns: -71.827851


Regardless of the reward function, a random policy outperforms epsilon greedy on this problem.
The reason why must be investigated.